In [ ]:
# Install PyMuPDF if not already installed
# Note: Run this notebook using `uv run jupyter notebook` to use the proper environment
import sys
import subprocess

try:
    import fitz
    print(f"✓ PyMuPDF already installed (version {fitz.version})")
except ImportError:
    print("Installing PyMuPDF with --user flag...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "--user", "pymupdf"])
    print("✓ PyMuPDF installed. Please restart the kernel and run this cell again.")

# PDF Table of Contents Generator

This notebook automatically generates a table of contents for PDF files with **two methods**:

## Primary Method: Extract from Existing TOC Page (NEW!)
1. **Detects** if PDF has a dedicated Table of Contents page
2. **Extracts** entries with hierarchy (based on indentation & font size)
3. **Matches** TOC entries to actual document locations
4. **Creates** bookmarks that mirror the TOC structure exactly

## Fallback Method: Font-Based Detection
If no TOC page is found, falls back to analyzing text formatting:
- Extracts text with font size and style information
- Automatically detects heading hierarchy based on font sizes
- Generates structured TOC with page numbers

## Features
- 📄 Prioritizes existing TOC pages for accuracy
- 🎯 Smart matching of TOC entries to document headings
- 🔖 Adds clickable bookmarks to PDF files
- 💾 Exports TOC in multiple formats (JSON, Markdown, Plain Text)
- 🎛️ Adjustable parameters for fine-tuning

## Requirements
This notebook uses UV for package management. Make sure you have UV installed.

## 1. Install and Import Required Libraries

We'll use PyMuPDF (fitz) for PDF processing as it provides excellent font information extraction capabilities.

In [ ]:
# Import required libraries
import fitz  # PyMuPDF
from collections import defaultdict, Counter
import re
from pathlib import Path
import json

print("Libraries imported successfully!")
print(f"PyMuPDF version: {fitz.version}")

## 2. Extract Text with Formatting Information

This function extracts all text blocks from the PDF along with their font metadata (size, name, flags like bold/italic).

## 2a. Detect and Extract Existing TOC Page

First, we'll try to find if the PDF already has a Table of Contents page and extract it.

In [ ]:
def detect_toc_page(pdf_path, max_pages_to_check=10):
    """
    Detect if PDF has an existing Table of Contents page(s).
    
    Args:
        pdf_path: Path to PDF file
        max_pages_to_check: How many pages from the start to check
        
    Returns:
        List of page numbers (0-indexed) with TOC content, or None if not found
    """
    doc = fitz.open(pdf_path)
    toc_patterns = [
        r'\btable\s+of\s+contents\b',
        r'\bcontents\b',
        r'\btoc\b'
    ]
    
    toc_pages = []
    toc_started = False
    
    for page_num in range(min(max_pages_to_check, len(doc))):
        page = doc[page_num]
        text = page.get_text().lower()
        
        # Check if page contains TOC-like patterns or "Chapter" headers
        for pattern in toc_patterns:
            if re.search(pattern, text, re.IGNORECASE):
                toc_started = True
                toc_pages.append(page_num)
                break
        
        # If TOC started, check if this page continues it
        if toc_started and page_num not in toc_pages:
            # Check for chapter headings or multiple page number references
            has_chapters = len(re.findall(r'\bchapter\s+\d+', text, re.IGNORECASE)) > 0
            page_refs = len(re.findall(r'\bpage\s+\d+|\.\.\.\s*\d+|\s+\d+\s*$', text, re.MULTILINE | re.IGNORECASE))
            
            if has_chapters or page_refs > 3:
                toc_pages.append(page_num)
            else:
                # TOC seems to have ended
                break
    
    doc.close()
    
    if toc_pages:
        if len(toc_pages) == 1:
            print(f"✓ Found TOC on page {toc_pages[0] + 1}")
        else:
            print(f"✓ Found TOC spanning pages {toc_pages[0] + 1}-{toc_pages[-1] + 1}")
        return toc_pages
    
    print("ℹ No existing TOC page detected")
    return None


def extract_toc_from_page(pdf_path, toc_page_nums):
    """
    Extract TOC entries from dedicated TOC page(s).
    
    Args:
        pdf_path: Path to PDF file
        toc_page_nums: List of page numbers (0-indexed) containing the TOC
        
    Returns:
        List of dictionaries with TOC entries
    """
    doc = fitz.open(pdf_path)
    
    # Handle both single page number and list
    if isinstance(toc_page_nums, int):
        toc_page_nums = [toc_page_nums]
    
    toc_entries = []
    pending_header = None  # Track chapter headers without page numbers
    
    for toc_page_num in toc_page_nums:
        page = doc[toc_page_num]
        
        # Get text with formatting
        blocks = page.get_text("dict", flags=11)["blocks"]
        
        for block in blocks:
            if block["type"] == 0:  # Text block
                for line in block["lines"]:
                    line_text = ""
                    line_x = float('inf')  # Track leftmost position for indentation
                    font_size = 0
                    
                    for span in line["spans"]:
                        line_text += span["text"]
                        line_x = min(line_x, span["bbox"][0])
                        font_size = max(font_size, span["size"])
                    
                    line_text = line_text.strip()
                    
                    # Skip empty lines and the "Table of Contents" header itself
                    if not line_text or re.match(r'^(table\s+of\s+)?contents$', line_text, re.IGNORECASE):
                        continue
                    
                    # Try to extract page number from end of line
                    # Patterns: "Text ... 123", "Text ... Page 123", "Text    123"
                    page_match = re.search(r'[.\s]+(?:page\s+)?(\d+)\s*$', line_text, re.IGNORECASE)
                    
                    if page_match:
                        page_number = int(page_match.group(1))
                        # Remove page number and dots from title
                        title = re.sub(r'[.\s]+(?:page\s+)?\d+\s*$', '', line_text, flags=re.IGNORECASE).strip()
                        title = re.sub(r'\.{2,}', '', title).strip()  # Remove dot leaders
                        
                        if title:
                            # If we have a pending header (chapter without page #), add it first
                            if pending_header:
                                toc_entries.append({
                                    "title": pending_header["title"],
                                    "page": page_number,  # Use page of first sub-entry
                                    "indent": pending_header["indent"],
                                    "font_size": pending_header["font_size"],
                                    "original_text": pending_header["original_text"],
                                    "is_chapter_header": True
                                })
                                pending_header = None
                            
                            # Add the current entry
                            toc_entries.append({
                                "title": title,
                                "page": page_number,
                                "indent": line_x,
                                "font_size": round(font_size, 2),
                                "original_text": line_text
                            })
                    else:
                        # No page number - likely a chapter header
                        # Look for "Chapter X:" patterns or larger font
                        if re.match(r'chapter\s+\d+', line_text, re.IGNORECASE) or font_size > 14.5:
                            # Clean up title (remove tabs, extra spaces)
                            title = re.sub(r'\s+', ' ', line_text).strip()
                            title = re.sub(r':\s*$', '', title)  # Remove trailing colon
                            
                            pending_header = {
                                "title": title,
                                "indent": line_x,
                                "font_size": round(font_size, 2),
                                "original_text": line_text
                            }
    doc.close()
    
    if toc_entries:
        print(f"✓ Extracted {len(toc_entries)} entries from TOC page")
    
    return toc_entries


def assign_toc_hierarchy(toc_entries):
    """
    Assign hierarchy levels based on indentation and font size.
    
    Args:
        toc_entries: List of TOC entry dictionaries
        
    Returns:
        List of TOC entries with 'level' field added
    """
    if not toc_entries:
        return []
    
    # Sort unique indents and font sizes
    indents = sorted(set(entry["indent"] for entry in toc_entries))
    font_sizes = sorted(set(entry["font_size"] for entry in toc_entries), reverse=True)
    
    # Create scoring system: indent + font size
    # Lower indent (left) + larger font = higher priority (lower level number)
    for entry in toc_entries:
        indent_score = indents.index(entry["indent"])
        font_score = font_sizes.index(entry["font_size"])
        entry["hierarchy_score"] = indent_score + (font_score * 0.5)
    
    # Sort by hierarchy score and assign levels
    scores = sorted(set(entry["hierarchy_score"] for entry in toc_entries))
    
    for entry in toc_entries:
        entry["level"] = scores.index(entry["hierarchy_score"]) + 1
    
    return toc_entries

# Example usage:
# toc_page = detect_toc_page("sample.pdf")
# if toc_page is not None:
#     entries = extract_toc_from_page("sample.pdf", toc_page)
#     entries = assign_toc_hierarchy(entries)

## 2b. Match TOC Entries to Actual Document Headings

Find where TOC entries actually appear in the document.

In [ ]:
def find_heading_in_document(pdf_path, heading_title, expected_page, search_window=5):
    """
    Find where a heading actually appears in the document.
    
    Args:
        pdf_path: Path to PDF
        heading_title: Title to search for
        expected_page: Expected page from TOC (1-indexed)
        search_window: How many pages around expected_page to search
        
    Returns:
        Actual page number (1-indexed) or expected_page if not found
    """
    doc = fitz.open(pdf_path)
    
    # Normalize the heading for comparison
    normalized_title = re.sub(r'\s+', ' ', heading_title.lower().strip())
    normalized_title = re.sub(r'[^\w\s]', '', normalized_title)  # Remove punctuation
    
    # Search around the expected page
    start_page = max(0, expected_page - search_window)
    end_page = min(len(doc), expected_page + search_window)
    
    best_match_page = expected_page
    best_match_score = 0
    
    for page_num in range(start_page, end_page):
        page = doc[page_num]
        text = page.get_text().lower()
        
        # Normalize page text
        normalized_text = re.sub(r'\s+', ' ', text)
        normalized_text = re.sub(r'[^\w\s]', '', normalized_text)
        
        # Check for exact match
        if normalized_title in normalized_text:
            doc.close()
            return page_num + 1  # Return 1-indexed
        
        # Check for partial match (fuzzy)
        title_words = set(normalized_title.split())
        text_words = set(normalized_text.split())
        match_score = len(title_words & text_words) / len(title_words) if title_words else 0
        
        if match_score > best_match_score:
            best_match_score = match_score
            best_match_page = page_num + 1
    
    doc.close()
    
    # If we found a good match (>70% word overlap), use it
    if best_match_score > 0.7:
        return best_match_page
    
    # Otherwise, trust the TOC page number
    return expected_page


def match_toc_to_document(pdf_path, toc_entries):
    """
    Match TOC entries to actual locations in the document.
    
    Args:
        pdf_path: Path to PDF
        toc_entries: List of TOC entries from extract_toc_from_page
        
    Returns:
        List of matched TOC entries with verified page numbers
    """
    matched_entries = []
    
    print(f"\nMatching {len(toc_entries)} TOC entries to document...")
    
    for i, entry in enumerate(toc_entries):
        actual_page = find_heading_in_document(
            pdf_path, 
            entry["title"], 
            entry["page"]
        )
        
        matched_entry = {
            "title": entry["title"],
            "page": actual_page,
            "level": entry["level"],
            "toc_page": entry["page"]  # Keep original for reference
        }
        
        matched_entries.append(matched_entry)
        
        if (i + 1) % 10 == 0:
            print(f"  Processed {i + 1}/{len(toc_entries)} entries...")
    
    print(f"✓ Matched all entries to document pages")
    
    return matched_entries

# Example usage:
# matched = match_toc_to_document("sample.pdf", toc_entries)

In [ ]:
def extract_text_with_formatting(pdf_path):
    """
    Extract text from PDF with font size, style, and position information.
    
    Args:
        pdf_path: Path to the PDF file
        
    Returns:
        List of dictionaries containing text blocks with their formatting info
    """
    doc = fitz.open(pdf_path)
    text_blocks = []
    page_count = len(doc)  # Store page count before closing
    
    for page_num in range(page_count):
        page = doc[page_num]
        
        # Extract text with detailed formatting information
        blocks = page.get_text("dict", flags=11)["blocks"]
        
        for block in blocks:
            if block["type"] == 0:  # Text block
                for line in block["lines"]:
                    for span in line["spans"]:
                        text = span["text"].strip()
                        if text:  # Only include non-empty text
                            text_blocks.append({
                                "text": text,
                                "page": page_num + 1,  # 1-indexed page numbers
                                "font_size": round(span["size"], 2),
                                "font_name": span["font"],
                                "flags": span["flags"],  # Font flags (bold, italic, etc.)
                                "is_bold": bool(span["flags"] & 2**4),  # Check bold flag
                                "is_italic": bool(span["flags"] & 2**1),  # Check italic flag
                                "color": span["color"],
                                "bbox": span["bbox"]  # Bounding box
                            })
    
    doc.close()
    print(f"Extracted {len(text_blocks)} text blocks from {page_count} pages")
    return text_blocks

# Example usage (you'll need to provide your PDF path)
# text_blocks = extract_text_with_formatting("sample.pdf")

## 3. Analyze Font Sizes and Styles

Analyze the distribution of font sizes to identify what's normal body text vs. headings.

In [ ]:
def analyze_font_statistics(text_blocks):
    """
    Analyze font sizes and styles to determine body text vs. headings.
    
    Args:
        text_blocks: List of text block dictionaries
        
    Returns:
        Dictionary with font statistics
    """
    font_sizes = [block["font_size"] for block in text_blocks]
    font_size_counter = Counter(font_sizes)
    
    # Find the most common font size (likely body text)
    most_common_sizes = font_size_counter.most_common(10)
    body_text_size = most_common_sizes[0][0] if most_common_sizes else 12.0
    
    # Get unique font sizes sorted in descending order
    unique_sizes = sorted(set(font_sizes), reverse=True)
    
    stats = {
        "body_text_size": body_text_size,
        "unique_sizes": unique_sizes,
        "size_distribution": dict(most_common_sizes),
        "min_size": min(font_sizes) if font_sizes else 0,
        "max_size": max(font_sizes) if font_sizes else 0,
    }
    
    print(f"Body text size (most common): {body_text_size}")
    print(f"Font size range: {stats['min_size']} - {stats['max_size']}")
    print(f"Unique font sizes: {len(unique_sizes)}")
    print("\nTop 10 font sizes by frequency:")
    for size, count in most_common_sizes:
        print(f"  {size}pt: {count} occurrences")
    
    return stats

# Example usage:
# stats = analyze_font_statistics(text_blocks)

## 4. Identify Heading Hierarchy

Classify text blocks as headings based on font size, boldness, and other heuristics.

In [ ]:
def identify_headings(text_blocks, stats, min_heading_size_ratio=1.1):
    """
    Identify potential headings based on font size and formatting.
    
    Args:
        text_blocks: List of text block dictionaries
        stats: Font statistics from analyze_font_statistics
        min_heading_size_ratio: Minimum ratio above body text to be considered a heading
        
    Returns:
        List of heading dictionaries with hierarchy levels
    """
    body_size = stats["body_text_size"]
    headings = []
    
    # Additional heuristics for identifying headings
    def is_likely_heading(block):
        """Check if a text block is likely a heading."""
        text = block["text"]
        size = block["font_size"]
        
        # Must be larger than body text
        if size < body_size * min_heading_size_ratio:
            return False
        
        # Length heuristics (headings are usually shorter)
        if len(text) > 200:  # Too long for a heading
            return False
        
        # Check if it looks like a sentence (ends with period)
        # Headings typically don't end with periods
        if len(text) > 50 and text.endswith('.'):
            return False
        
        # Prefer bold text for headings
        if block["is_bold"]:
            return True
        
        # If significantly larger than body text, likely a heading
        if size >= body_size * 1.3:
            return True
        
        return False
    
    # Filter potential headings
    potential_headings = [block for block in text_blocks if is_likely_heading(block)]
    
    # Determine heading levels based on font sizes
    if potential_headings:
        # Get unique sizes from potential headings
        heading_sizes = sorted(set([h["font_size"] for h in potential_headings]), reverse=True)
        
        # Create size-to-level mapping (larger = higher level)
        size_to_level = {}
        for idx, size in enumerate(heading_sizes[:6]):  # Support up to 6 levels
            size_to_level[size] = idx + 1
        
        # Assign levels to headings
        for block in potential_headings:
            level = size_to_level.get(block["font_size"], 3)
            headings.append({
                "text": block["text"],
                "page": block["page"],
                "level": level,
                "font_size": block["font_size"],
                "is_bold": block["is_bold"]
            })
    
    print(f"Identified {len(headings)} potential headings")
    if headings:
        level_counts = Counter([h["level"] for h in headings])
        for level in sorted(level_counts.keys()):
            print(f"  Level {level} (H{level}): {level_counts[level]} headings")
    
    return headings

# Example usage:
# headings = identify_headings(text_blocks, stats)

## 5. Generate Table of Contents Structure

Create a hierarchical TOC structure from the identified headings.

In [ ]:
def generate_toc_structure(headings):
    """
    Generate a hierarchical table of contents structure.
    
    Args:
        headings: List of heading dictionaries
        
    Returns:
        List of TOC entries in PyMuPDF outline format
    """
    if not headings:
        return []
    
    # PyMuPDF has strict rules:
    # 1. First entry must be level 1
    # 2. Levels can only increase by 1 at a time
    # 3. Levels can decrease by any amount
    
    first_level = headings[0]["level"]
    level_offset = 1 - first_level
    
    toc = []
    prev_level = 0
    
    for heading in headings:
        # Apply offset
        normalized_level = heading["level"] + level_offset
        normalized_level = max(1, normalized_level)
        
        # Ensure we don't jump more than 1 level up from previous
        if normalized_level > prev_level + 1:
            normalized_level = prev_level + 1
        
        toc.append([
            normalized_level,
            heading["text"],
            heading["page"]
        ])
        
        prev_level = normalized_level
    
    return toc


def format_toc_display(toc):
    """
    Format TOC for human-readable display.
    
    Args:
        toc: List of TOC entries
        
    Returns:
        Formatted string representation of TOC
    """
    output = []
    output.append("=" * 80)
    output.append("TABLE OF CONTENTS")
    output.append("=" * 80)
    output.append("")
    
    for entry in toc:
        level, title, page = entry[0], entry[1], entry[2]
        indent = "  " * (level - 1)
        output.append(f"{indent}{'•' if level > 1 else '▪'} {title} ... Page {page}")
    
    output.append("")
    output.append("=" * 80)
    
    return "\n".join(output)


def export_toc_formats(toc, base_filename="toc"):
    """
    Export TOC in multiple formats.
    
    Args:
        toc: List of TOC entries
        base_filename: Base name for export files
    """
    # JSON format
    json_data = [
        {"level": entry[0], "title": entry[1], "page": entry[2]}
        for entry in toc
    ]
    json_path = f"{base_filename}.json"
    with open(json_path, "w", encoding="utf-8") as f:
        json.dump(json_data, f, indent=2, ensure_ascii=False)
    print(f"Exported JSON: {json_path}")
    
    # Markdown format
    md_lines = ["# Table of Contents\n"]
    for entry in toc:
        level, title, page = entry[0], entry[1], entry[2]
        indent = "  " * (level - 1)
        md_lines.append(f"{indent}- {title} (Page {page})")
    
    md_path = f"{base_filename}.md"
    with open(md_path, "w", encoding="utf-8") as f:
        f.write("\n".join(md_lines))
    print(f"Exported Markdown: {md_path}")
    
    # Plain text format
    txt_path = f"{base_filename}.txt"
    with open(txt_path, "w", encoding="utf-8") as f:
        f.write(format_toc_display(toc))
    print(f"Exported Text: {txt_path}")

# Example usage:
# toc = generate_toc_structure(headings)
# print(format_toc_display(toc))
# export_toc_formats(toc)

## 6. Add TOC Bookmarks to PDF

Add the generated TOC as clickable bookmarks (outline) to the PDF file.

In [ ]:
def add_toc_to_pdf(input_pdf_path, output_pdf_path, toc):
    """
    Add table of contents as bookmarks to a PDF file.
    
    Args:
        input_pdf_path: Path to input PDF
        output_pdf_path: Path to output PDF with TOC
        toc: List of TOC entries in PyMuPDF format
    """
    doc = fitz.open(input_pdf_path)
    
    # Set the table of contents
    doc.set_toc(toc)
    
    # Save the modified PDF
    doc.save(output_pdf_path, garbage=4, deflate=True)
    doc.close()
    
    print(f"✓ Added {len(toc)} bookmarks to PDF")
    print(f"✓ Saved output to: {output_pdf_path}")
    
    # Verify the TOC was added
    verify_doc = fitz.open(output_pdf_path)
    saved_toc = verify_doc.get_toc()
    verify_doc.close()
    
    if saved_toc:
        print(f"✓ Verification: PDF now contains {len(saved_toc)} bookmark entries")
    else:
        print("⚠ Warning: No bookmarks found in output PDF")

# Example usage:
# add_toc_to_pdf("input.pdf", "output_with_toc.pdf", toc)

## 7. Complete Workflow Function

Combine all steps into a single workflow function for easy use.

In [ ]:
def generate_pdf_toc(input_pdf_path, output_pdf_path=None, 
                     export_formats=True, min_heading_ratio=1.1,
                     display_toc=True, use_existing_toc=True,
                     flat_structure=False):
    """
    Complete workflow to generate TOC for a PDF.
    
    Args:
        input_pdf_path: Path to input PDF file
        output_pdf_path: Path for output PDF (default: adds '_with_toc' suffix)
        export_formats: Whether to export TOC in JSON, MD, TXT formats
        min_heading_ratio: Minimum font size ratio to identify headings
        display_toc: Whether to print the TOC to console
        use_existing_toc: Whether to try extracting from existing TOC page first
        flat_structure: If True, all bookmarks at level 1 (no hierarchy/indentation)
        
    Returns:
        Dictionary with TOC and statistics
    """
    print("=" * 80)
    print("PDF TABLE OF CONTENTS GENERATOR")
    print("=" * 80)
    print(f"\nProcessing: {input_pdf_path}\n")
    
    # Set default output path
    if output_pdf_path is None:
        path = Path(input_pdf_path)
        output_pdf_path = str(path.parent / f"{path.stem}_with_toc{path.suffix}")
    
    headings = None
    stats = None
    method_used = None
    toc_page_nums = None  # Store TOC page numbers for bookmarking
    
    # Try to extract from existing TOC page first
    if use_existing_toc:
        print("Step 1: Checking for existing TOC page...")
        toc_page = detect_toc_page(input_pdf_path)
        print()
        
        if toc_page is not None:
            toc_page_nums = toc_page if isinstance(toc_page, list) else [toc_page]
            print("Step 2: Extracting TOC from dedicated page...")
            toc_entries = extract_toc_from_page(input_pdf_path, toc_page)
            
            if toc_entries:
                print("Step 3: Assigning hierarchy to TOC entries...")
                toc_entries = assign_toc_hierarchy(toc_entries)
                
                print("Step 4: Matching TOC entries to document pages...")
                matched_entries = match_toc_to_document(input_pdf_path, toc_entries)
                
                # Convert to headings format for compatibility
                headings = matched_entries
                method_used = "existing_toc"
                print(f"✓ Using TOC page method: {len(headings)} entries\n")
    
    # Fallback to font-based detection if no TOC found
    if headings is None:
        print("Step 1: Extracting text with formatting...")
        text_blocks = extract_text_with_formatting(input_pdf_path)
        print()
        
        print("Step 2: Analyzing font statistics...")
        stats = analyze_font_statistics(text_blocks)
        print()
        
        print("Step 3: Identifying headings...")
        headings = identify_headings(text_blocks, stats, min_heading_ratio)
        print()
        
        if not headings:
            print("⚠ Warning: No headings found. Try adjusting min_heading_ratio parameter.")
            return None
        
        method_used = "font_detection"
        print(f"✓ Using font detection method: {len(headings)} entries\n")
    
    # Generate TOC structure
    print(f"Step {5 if method_used == 'existing_toc' else 4}: Generating TOC structure...")
    
    # For existing TOC, convert to PyMuPDF format
    if method_used == "existing_toc":
        toc = []
        
        # Add TOC page itself as first bookmark if we found one
        if toc_page_nums:
            # Use the first TOC page for the bookmark (1-indexed for display)
            toc.append([1, "Table of Contents", toc_page_nums[0] + 1])
        
        for entry in headings:
            toc.append([
                entry["level"] + 1 if toc_page_nums else entry["level"],  # Shift levels if TOC bookmark added
                entry["title"],
                entry["page"]
            ])
        # Ensure proper hierarchy
        toc = normalize_toc_hierarchy(toc)
    else:
        toc = generate_toc_structure(headings)
    
    # Apply flat structure if requested
    if flat_structure:
        print("✓ Converting to flat structure (all entries at level 1)")
        toc = [[1, entry[1], entry[2]] for entry in toc]
    
    print()
    
    # Display TOC
    if display_toc:
        print(format_toc_display(toc))
        print()
    
    # Export formats
    if export_formats:
        step_num = 6 if method_used == "existing_toc" else 5
        print(f"Step {step_num}: Exporting TOC formats...")
        base_name = Path(input_pdf_path).stem
        export_toc_formats(toc, base_name)
        print()
    add_toc_to_pdf(input_pdf_path, output_pdf_path, toc)
    # Add TOC to PDF
    step_num = 7 if method_used == "existing_toc" else 6
    print(f"Step {step_num}: Adding bookmarks to PDF...")
    add_toc_to_pdf(input_pdf_path, output_pdf_path, toc)
    print()
    
    print("=" * 80)
    print(f"✓ COMPLETE! (Method: {method_used})")
    print("=" * 80)
    
    return {
        "toc": toc,
        "headings": headings,
        "stats": stats,
        "method": method_used,
        "output_path": output_pdf_path
    }


def normalize_toc_hierarchy(toc):
    """
    Ensure TOC hierarchy follows PyMuPDF rules.
    
    Args:
        toc: List of [level, title, page] entries
        
    Returns:
        Normalized TOC list
    """
    if not toc:
        return []
    
    normalized = []
    prev_level = 0
    
    for entry in toc:
        level, title, page = entry[0], entry[1], entry[2]
        
        # Ensure first entry is level 1
        if not normalized:
            level = 1
        # Ensure levels don't jump more than 1
        elif level > prev_level + 1:
            level = prev_level + 1
        # Ensure level is at least 1
        level = max(1, level)
        
        normalized.append([level, title, page])
    
        prev_level = level

# Example usage:# result = generate_pdf_toc("sample.pdf")
    return normalized


    return normalized

    # result = generate_pdf_toc("sample.pdf")# Example usage:

## 8. Test with Your PDF

Now you can test the generator with your own PDF file. Simply provide the path to your PDF below.

In [ ]:
# Replace with your PDF path
pdf_path = "your_document.pdf"

# Generate TOC with default settings
result = generate_pdf_toc(pdf_path)

# Advanced usage with custom parameters (commented out - uncomment to use):
# result = generate_pdf_toc(
#     pdf_path,
#     output_pdf_path="custom_output.pdf",
#     min_heading_ratio=1.15,  # Adjust if too many/few headings detected
#     export_formats=True,
#     display_toc=True,
#     use_existing_toc=True,  # Set to False to force font-based detection
#     flat_structure=False  # Set to True for flat TOC (no hierarchy/indentation)
# )